In [2]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from os import path as op
import itertools
import re
import xarray as xr
from sklearn.model_selection import train_test_split
import numpy.random as nr

%matplotlib inline

/scratch/adomakor412/conda/envs/MyEnv/lib/python3.6/site-packages/dask/config.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
nr.seed = 42

In [4]:
inputPath = '/scratch/adomakor412/all_npy3'
PATH = os.path.expanduser(inputPath)
ncPath = os.path.expanduser('/scratch/adomakor412/april_data_2019')

In [5]:
columns = [
    "time",
    "longitude",
    "latitude",
    #"band",#Necessary?
    "G17_Temp",
    "G17_mean",
    "G17_std",#mean and std outside inner for loop for comp. eff.
    "target_G16_Temp"
]

df = pd.DataFrame({},columns = columns)

In [6]:
def longitude(lonMin, lonMax, col, colSize):
    lon = (col/colSize)* (lonMax - lonMin)
    return lon

def latitude(latMin, latMax, row, rowSize):
    lat = (row/rowSize)* (latMax - latMin)
    return lat

def Rad2BT(rad, planck_fk1, planck_fk2, planck_bc1, planck_bc2):
    """Radiances to Brightness Temprature (using black body equation)"""
    invRad = np.array(rad)**(-1)
    arg = (invRad*planck_fk1) + 1.0
    T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2) 
    return T

In [7]:
import logging
logger = logging.getLogger()

In [8]:
extent_pc = [-109.59326, -102.40674, 8.94659, -8.94656]

with open('logML', 'a') as log:   
    for (bb,dd) in list(itertools.product([8],[5])):#Let's start with one day
        DD = str(98+dd).zfill(3)
        lookup = f'M6C08_G17_s2019{DD}0000'
        ncFiles = [f for f in os.listdir(ncPath) if re.search(lookup,f)]
        #ncFiles = mySort(ncFiles)
        npFiles = [f for f in os.listdir( PATH ) if re.search(lookup,f)]
        #ncFiles = mySort(ncFiles)

        for ncf, npf in zip(ncFiles,npFiles):
            try:
                imageBox = np.load(op.join( PATH,npf))
                myFile = xr.open_dataset(op.join(ncPath,ncf))
                planck_fk1 = float(myFile['planck_fk1'].data)
                planck_fk2 = float(myFile['planck_fk2'].data) 
                planck_bc1 = float(myFile['planck_bc1'].data)                       
                planck_bc2 = float(myFile['planck_bc2'].data)

                time = ncf[31:38]
                G17_mean = Rad2BT(imageBox.mean(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)
                G17_std = Rad2BT(imageBox.std(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)

                hh = ncf[34:36]
                mm = ncf[36:38]
                
                print(ncf)
                print(str(ncf), file=log)
                logger.info(str(ncf))

                G16_npy = np.load( op.join(PATH, npf.replace('G16','G17',1)) )
                G16_ncf = xr.open_dataset(op.join( ncPath, ncf.replace('G16','G17',1) ))
                G16_fk1 = float(G16_ncf['planck_fk1'].data)
                G16_fk2 = float(G16_ncf['planck_fk2'].data) 
                G16_bc1 = float(G16_ncf['planck_bc1'].data)                       
                G16_bc2 = float(G16_ncf['planck_bc2'].data)

                target_G16_Temp = Rad2BT(G16_npy, G16_fk1, G16_fk2, G16_bc1, G16_bc2)
                x,y = imageBox.shape[0],imageBox.shape[1]
                for i,j in itertools.product(range(x),range(y)):
                    lon = longitude( extent_pc[0], extent_pc[1], i, x )
                    lat = latitude( extent_pc[2], extent_pc[3], j, y)

                    G17_Temp = Rad2BT(imageBox[i,j], planck_fk1, planck_fk2, planck_bc1, planck_bc2)#unfiltered
                    target_G16_Temp = Rad2BT( G16_npy[i,j], G16_fk1, G16_fk2, G16_bc1, G16_bc2 )
                    
                    row = [time, lon, lat, G17_Temp, G17_mean, G17_std, target_G16_Temp]

                    df = df.append(pd.DataFrame([row],columns=columns),ignore_index=True)
                    
            except ValueError as e:
                logger.exception(e)
                print(e)
                print(e, file=log)
                continue 
    
# df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", names=columns)
# for column in df.columns:
#     if df[column].dtype == "object":
#         df[column] = df[column].str.strip()
# df = df.dropna()

OR_ABI-L1b-RadF-M6C08_G17_s20191030000339_e20191030009405_c20191030009441.nc


In [1]:
from sklearn.neural_network import MLPRegressor
#from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [10]:
print(len(df))
print(df.head())

401401
      time  longitude  latitude    G17_Temp    G17_mean     G17_std  \
0  1030000        0.0 -0.000000  242.868317  243.715676  202.188943   
1  1030000        0.0 -0.044621  242.919616  243.715676  202.188943   
2  1030000        0.0 -0.089243  242.919616  243.715676  202.188943   
3  1030000        0.0 -0.133864  242.919616  243.715676  202.188943   
4  1030000        0.0 -0.178485  242.919616  243.715676  202.188943   

   target_G16_Temp  
0       242.868317  
1       242.919616  
2       242.919616  
3       242.919616  
4       242.919616  


In [20]:
dfSample = df.dropna().sample(n=1000)
X = dfSample.drop(["target_G16_Temp"],axis=1).astype('float')#try as float
y = dfSample.drop(columns[:-1],axis=1).astype('float')

In [21]:
X.head()

,time,longitude,latitude,G17_Temp,G17_mean,G17_std
109589,1030000.0,1.959960,-5.176073,242.088872,243.715676,202.188943
285933,1030000.0,5.118870,-0.892426,243.327095,243.715676,202.188943
276537,1030000.0,4.946566,-11.066088,243.225705,243.715676,202.188943
136657,1030000.0,2.440976,-14.144959,244.519748,243.715676,202.188943
356858,1030000.0,6.382434,-16.465268,240.136709,243.715676,202.188943


In [22]:
y.head()

,target_G16_Temp
109589,242.088872
285933,243.327095
276537,243.225705
136657,244.519748
356858,240.136709


In [28]:
from sklearn.metrics import mean_squared_error
X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(y),random_state=1,test_size=0.1)
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)

In [29]:
X_test[:5]

,time,longitude,latitude,G17_Temp,G17_mean,G17_std
267830,1030000.0,4.788620,-16.197540,245.243669,243.715676,202.188943
32946,1030000.0,0.588706,-2.855765,242.970832,243.715676,202.188943
205193,1030000.0,3.668643,-12.583213,247.009727,243.715676,202.188943
325918,1030000.0,5.829625,-13.654124,240.855590,243.715676,202.188943
273331,1030000.0,4.889131,-11.155330,244.176372,243.715676,202.188943


In [30]:
regr.predict(X_test[:5])

array([243.75886052, 243.07278366, 243.16213075, 244.16058273,
       243.40035261])

In [31]:
regr.score(X_test, y_test)

-0.4281456898253139

In [34]:
prediction = regr.predict(X_test)

In [35]:
errors = abs(prediction - y_test)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'Kelvin')
print('Mean Square Error:', mean_squared_error(prediction, y_test))

Mean Absolute Error: 3.02 Kelvin
Mean Square Error: 23.9570997752824
